In [1]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();

await history.addMessage(new HumanMessage("hi"));
await history.addMessage(new AIMessage("What can I do for you?"));

const messages = await history.getMessages();

console.log(messages);

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "hi", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What can I do for you?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What can I do for you?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [3]:
import { load } from "dotenv";
const env = await load();
const process = {
    env
}

In [4]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";

const chatModel = new ChatAlibabaTongyi({
    model: "qwen-turbo", // Available models: qwen-turbo, qwen-plus, qwen-max
    temperature: 1,
});

const prompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.
    You are talkative and provides lots of specific details from its context. 
    If the you does not know the answer to a question, it truthfully says you do not know.`],
    new MessagesPlaceholder("history_message"),
]);

const chain = prompt.pipe(chatModel);

In [7]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

// process.env.LANGCHAIN_VERBOSE = "false";

const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("hi, my name is Kai"));

const res1 = await chain.invoke({
    history_message: await history.getMessages()
});

console.log(res1);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello Kai! Nice to meet you. Is there anything specific you'd like to chat about or ask? I'm here to"... 53 more characters,
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello Kai! Nice to meet you. Is there anything specific you'd like to chat about or ask? I'm here to"... 53 more characters,
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 73, completionTokens: 34, totalTokens: 107 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}


In [8]:
await history.addMessage(res1)
await history.addMessage(new HumanMessage("What is my name?"));

In [9]:
const res2 = await chain.invoke({
    history_message: await history.getMessages()
});
console.log(res2);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Your name is Kai.",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Your name is Kai.",
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 122, completionTokens: 5, totalTokens: 127 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}


In [11]:
import { Ollama } from "@langchain/community/llms/ollama";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

const chatModel = new Ollama({
  baseUrl: "http://localhost:11434", 
  model: "qwen:7b", 
});

const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant. Answer all questions to the best of your ability."],
    new MessagesPlaceholder("history_message"),
    ["human","{input}"]
]);

const history = new ChatMessageHistory();
const chain = prompt.pipe(chatModel)

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: (_sessionId) => history,
  inputMessagesKey: "input",
  historyMessagesKey: "history_message",
});

In [12]:
const res1 = await chainWithHistory.invoke({
    input: "hi, my name is Kai"
},{
    configurable: { sessionId: "none" }
});
console.log(res1);

Hello Kai! Nice to meet you. How can I assist you today?


In [13]:
const res2 = await chainWithHistory.invoke({
    input: "我的名字叫什么？"
},{
    configurable: { sessionId: "none" }
});
console.log(res2);

您的名字是Kai。


In [15]:
const msg = await history.getMessages();
console.log(msg);

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "hi, my name is Kai",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi, my name is Kai",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hello Kai! Nice to meet you. How can I assist you today?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello Kai! Nice to meet you. How can I assist you today?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "我的名字叫什么？",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "我的名字叫什么？",
    name: undefined,
    additional_kwar

In [16]:
import { Ollama } from "@langchain/community/llms/ollama";
import { RunnableSequence } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const summaryModel = new Ollama({
  baseUrl: "http://localhost:11434", 
  model: "qwen:7b", 
});

const summaryPrompt = ChatPromptTemplate.fromTemplate(`
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
`); 

const summaryChain = RunnableSequence.from([
    summaryPrompt,
    summaryModel,
    new StringOutputParser(),
])

In [23]:
const newSummary = await summaryChain.invoke({
    summary: "",
    new_lines: "I'm 18"
});

In [24]:
await summaryChain.invoke({
    summary: newSummary,
    new_lines: "I'm male"
});

"At 18 years old, the individual shares their age and gender."

In [25]:
const chatModel = new Ollama({
  baseUrl: "http://localhost:11434", 
  model: "qwen:7b", 
});
const chatPrompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.

    Here is the chat history summary:
    {history_summary}
    `],
    ["human","{input}"]
]);
let summary = ""
const history = new ChatMessageHistory();

In [30]:
import { RunnablePassthrough } from "@langchain/core/runnables";
import { getBufferString } from "@langchain/core/messages";

const chatChain = RunnableSequence.from([
    {
        input: new RunnablePassthrough({
             func: (input) => history.addUserMessage(input)
        })
    },
    RunnablePassthrough.assign({
        history_summary: () => summary
    }),
    chatPrompt,
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (input) => {
            history.addAIChatMessage(input)
            const messages = await history.getMessages()
            const new_lines = getBufferString(messages)
            const newSummary = await summaryChain.invoke({
                summary,
                new_lines
            })
            history.clear()
            summary = newSummary      
        }
    })
])

In [31]:
import { RunnableMap } from "@langchain/core/runnables";

const mapChain = RunnableMap.from({
    a: () => "a",
    b: () => "b"
});

const res = await mapChain.invoke();
console.log(res);


{ a: "a", b: "b" }


In [32]:
await chatChain.invoke("我现在饿了")

"我理解您感到饥饿。如果您在家中，可能需要准备一些食物来满足饥饿感。比如，您可以吃些坚果、水果，或者做一些简单的饭菜。\n" +
  "\n" +
  "如果在外面或者不方便自己准备，可以考虑附近是否有餐厅或便利店提供食品服务。也可以"... 41 more characters

In [34]:
const res = await chatChain.invoke("我今天想吃方便面");
console.log(res);

如果你想吃方便面，可以按照以下方法来解决：

1. **确认库存**：首先检查家里是否还有剩余的方便面。如果有足够的量，可以直接使用。

2. **购买新包装**：如果家中没有方便面，需要去附近的便利店或超市购买新的方便面包装。

3. **准备泡面**：在买到方便面后，拆开包装并通常需要用热水来煮面。注意水温不要过高，以免破坏面条的口感。

4. **添加调料**：煮好的方便面可以根据个人口味添加调料，如酱油、油辣酱、蔬菜等。

按照这些步骤，你就可以轻松享用一碗美味的方便面了。
